In [36]:
# import libraries
import numpy as np 
import pandas as pd
import string

In [37]:
# read the CSVs
df = pd.read_csv('train.csv')
df1 = pd.read_csv('test.csv')

In [38]:
# set tweetid as index
df_new = df1.set_index('tweetid')

In [39]:
# clean the train csv
df_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['message'] = df['message'].replace(to_replace = df_url, value = subs_url, regex = True)
df['message'] = df['message'].str.lower()

In [40]:
# function to remove punctuations
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [41]:
df['message'] = df['message'].apply(remove_punctuation)

In [42]:
# clean the test csv
df_test = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_urltest = r'url-web'
df_new['message'] = df_new['message'].replace(to_replace = df_test, value = subs_urltest, regex = True)
df_new['message'] = df_new['message'].str.lower()

In [43]:
# function to remove punctuations
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [44]:
df_new['message'] = df_new['message'].apply(remove_punctuation)

In [45]:
# model
from sklearn.model_selection import train_test_split

X = df['message']  
y = df['sentiment']
X_unseen = df_new['message'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('lr', LogisticRegression())
                      ])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)

In [48]:
predictions = text_clf.predict(X_test)

In [49]:
y_pred = text_clf.predict(X_unseen)

In [50]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 43  15  63   5]
 [  7  74 133  10]
 [  4  24 807  60]
 [  2   3  63 269]]


In [51]:
# classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.77      0.34      0.47       126
           0       0.64      0.33      0.44       224
           1       0.76      0.90      0.82       895
           2       0.78      0.80      0.79       337

    accuracy                           0.75      1582
   macro avg       0.74      0.59      0.63      1582
weighted avg       0.75      0.75      0.73      1582



In [52]:
# print accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7541087231352718


In [53]:
New = df_new.index

In [54]:
New = {'tweetid': New, 'sentiment': np.round(y_pred,0)}
submission = pd.DataFrame(data= New)
submission = submission[['tweetid', 'sentiment']]

In [55]:
submission.set_index('tweetid')

,sentiment
tweetid,
169760,1
35326,1
224985,1
476263,1
872928,0
...,...
895714,1
875167,1
78329,2
